In [1]:
import sys,os
import json
import numpy as np
import matplotlib.pyplot as plt
from fcc_python_tools.locations import loc
from fcc_python_tools import plotting
import pandas as pd
from root_pandas import read_root, to_root
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.externals import joblib
from sklearn.utils.class_weight import compute_sample_weight
from scipy.stats import ks_2samp
import xgboost as xgb

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Roman']})
rc('text', usetex=True)

Welcome to JupyROOT 6.20/06


/cvmfs/sft.cern.ch/lcg/views/LCG_97apython3/x86_64-centos7-gcc8-opt/lib/python3.7/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.18.5 is currently installed but you installed root_numpy against numpy 1.16.4. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)
/cvmfs/sft.cern.ch/lcg/views/LCG_97apython3/x86_64-centos7-gcc8-opt/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
#Load signal with all the BDTs applied
BDT_str = "Zuds_BDT_Zcc_BDT_Zbb_BDT_all_BDT"
path = f"{loc.OUT}/ROOT"
df_sig = read_root(f"{path}/Bc2TauNu_{BDT_str}.root","events")

In [5]:
#Load backgrounds with all BDTs applied
df_bkg = {}
for q in ["uds","cc","bb"]:
    df_bkg[q] = read_root(f"{path}/inclusive_Z{q}_{BDT_str}.root","events")

In [12]:
#Get maximum BDT scores for each background type
BDT_high = {}
for q in ["uds","cc","bb"]:
    BDT_high[q] = df_bkg[q]["BDT_all"].max()
#Find highest of these
max_BDT = 0.
for q in ["uds","cc","bb"]:
    if(BDT_high[q] > max_BDT):
        max_BDT = BDT_high[q]
print(max_BDT)

0.9950550198554993


In [14]:
#Cut on the maximum BDT score, which rejects all of the background in each sample i.e. 10^6 rejection of qq, cc, and bb
n_sig_tot = float(len(df_sig))
n_sig_pass = float(len(df_sig.query(f"BDT_all > {max_BDT}")))
eff_sig = n_sig_pass / n_sig_tot
print("Signal efficiency for BDT cut that rejects 10^6 of all three backgrounds: %.3f" % eff_sig)

Signal efficiency for BDT cut that rejects 10^6 of all three backgrounds: 0.600


In [15]:
#Cut on each background with its own MVA for comparison
BDT_type = {"uds": "qq",
            "cc": "cc",
            "bb": "bb"
            }
BDT_high = {}
for q in ["uds","cc","bb"]:
    BDT_high[q] = df_bkg[q][f"BDT_{BDT_type[q]}"].max()
print(BDT_high)

{'uds': 0.9978569746017456, 'cc': 0.9948310256004333, 'bb': 0.9901398122310638}


In [16]:
#Cut on the three BDTs in signal and check the efficiency
n_sig_pass = float(len(df_sig.query(f"BDT_qq > {BDT_high['uds']} and BDT_cc > {BDT_high['cc']} and BDT_bb > {BDT_high['bb']}")))
eff_sig = n_sig_pass / n_sig_tot
print("Signal efficiency for BDT cuts that rejects 10^6 of all three backgrounds: %.3f" % eff_sig)

Signal efficiency for BDT cuts that rejects 10^6 of all three backgrounds: 0.537
